In [5]:
import pandas as pd
import numpy as np 
import time 
from keras.models import load_model

test_file = 'data/redondo/test.csv'
df = pd.read_csv(test_file, encoding='utf-8').fillna(0)
df = np.array(df)
lstm = load_model("TrafficPrediction/lstm_model.h5")
scale_ = 0.005076142131979695

In [14]:
import time
n = df.shape[0]
lag = 12
start = time.time()
n = 100
for i in range(0, 100):
    data = df[i:i+lag]
    flows = data[:, 2]
    # Transform the data for prediction
    flows = np.asarray(flows)
    flows = np.multiply(flows, scale_)
    print(flows)
    flows = np.reshape(flows, (1, lag, 1))
#     print(flows)
    
    # Make the prediction
    preds = lstm.predict(flows)
    print(preds)
    time.sleep(2)
    
    # Inverse transform
    preds = np.divide(preds, scale_)
    if i % 10 == 0:
        print("processed {}".format(i))
#     print("Predictions : {}".format(preds))
    
    #time.sleep(0.25)
end = time.time()
print(end-start)

[0.583756345177665 0.24365482233502536 0.4111675126903553
 0.3705583756345177 0.36040609137055835 0.45685279187817257
 0.23350253807106597 0.26903553299492383 0.2081218274111675
 0.4111675126903553 0.34517766497461927 0.20304568527918782]
[[0.2735302]]


KeyboardInterrupt: 

In [7]:
print(lstm.get_layer("lstm_1").input)

Tensor("lstm_1_input_1:0", shape=(?, 12, 1), dtype=float32)


In [12]:
lstm.save_weights('/home/ben/git/research/TrafficPrediction/model_weights')
text_file = open("/home/ben/git/research/TrafficPrediction/model_json", "w")
s = lstm.to_json()
print(s)
text_file.write(s)

{"class_name": "Sequential", "config": [{"class_name": "LSTM", "config": {"name": "lstm_1", "trainable": true, "batch_input_shape": [null, 12, 1], "dtype": "float32", "return_sequences": true, "return_state": false, "go_backwards": false, "stateful": false, "unroll": false, "units": 256, "activation": "tanh", "recurrent_activation": "hard_sigmoid", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "recurrent_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "unit_forget_bias": true, "kernel_regularizer": null, "recurrent_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "recurrent_constraint": null, "bias_constraint": null, "dropout": 0.0, "recurrent_dropout": 0.0, "implementation": 

3152

In [32]:
from keras import backend as K

min_ = -0.03880597014925373
inp = lstm.input
outputs = [layer.output for layer in lstm.layers]
functors = [K.function([inp, K.learning_phase()], [out]) for out in outputs]

test_data = [223, 190, 218, 223, 202, 220, 214, 208, 131, 164, 176, 184]
test_data = np.asarray(test_data)
test_data = np.multiply(test_data, scale_)
test_data = np.add(test_data, min_)
test_data = np.reshape(test_data, (1, lag, 1))
layer_outs = [func([test_data, 1.]) for func in functors]
with open("/home/ben/git/research/TrafficPrediction/layer_outs", "w") as text_file:
    for layer in layer_outs:
        for out in layer:
            print(type(out))
            text_file.write(str(out) + " ")
        text_file.write("\n")

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
